In [1]:
from pathlib import Path
import tkinter as tk
from tkinter import  Canvas, Entry,  Button, PhotoImage
import math
import heapq
from collections import Counter
from decimal import Decimal, getcontext

## 01 Run-Length-Encoding

In [2]:
class RLEEncoder:
    def __init__(self, data):
        self.data = data
        self.counts = []
        self.encoded_seq = ""
        self.bits = 0
        self.vectors = 0

    def run_length_encoding(self):
        count = 1
        self.vectors = 1
        for n in range(1, len(self.data)):
            if self.data[n] == self.data[n-1]:
                count += 1
            else:
                self.encoded_seq += "({}, {})".format(self.data[n-1], count)
                self.counts.append(count)
                count = 1
                self.vectors += 1
        self.encoded_seq += "({}, {})".format(self.data[-1], count)
        self.counts.append(count)

    def calculate_character_probability(self):
        character_count = {}
        for char in self.data:
            if char in character_count:
                character_count[char] += 1
            else:
                character_count[char] = 1

        total_characters = len(self.data)

        character_probability = {}
        for char, count in character_count.items():
            probability = count / total_characters
            character_probability[char] = round(probability,2)

        return character_probability

    def calculate_entropy(self):
        character_probability = self.calculate_character_probability()

        entropy = 0
        for probability in character_probability.values():
            entropy -= probability * math.log2(probability)

        return entropy

    def calculate_metrics(self):
        self.run_length_encoding()
        original_message = self.data
        initial_bits = 0
        if set(self.data).issubset({'0', '1'}):  # Check if the data is binary
            self.bits=1
            initial_bits = len(self.data)
        elif all(c.isdigit() for c in self.data):
            self.bits = math.ceil(math.log2(abs(max(int(x) for x in self.data)) + 1))
            initial_bits = math.ceil(math.log2(abs(int(self.data)) + 1))
        else:
            self.bits = 8
            initial_bits = len(self.data) * self.bits

        final_bits = self.vectors * (self.bits + math.ceil(math.log2(max(self.counts) + 1)))
        compression_ratio = round(initial_bits / final_bits,3)
        probabilities = self.calculate_character_probability()
        entropy = round(self.calculate_entropy(),3)

        return {
            "Original Message": original_message,
            "Initial Bits": initial_bits,
            "Entropy": entropy,
            "Encoded Seq": self.encoded_seq,
            "Final Bits": final_bits,
            "Probabilities": probabilities,            
            "Compression Ratio": compression_ratio
        }

## 02 Huffman 

In [3]:
class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

class HuffmanCoding:
    def __init__(self, text):
        self.text = text
        self.tree = self.build_huffman_tree()
        self.codes = {}
        self.build_huffman_mapping(self.tree)

    def build_huffman_tree(self):
        frequency = Counter(self.text)
        priority_queue = [Node(char, freq) for char, freq in frequency.items()]
        heapq.heapify(priority_queue)

        while len(priority_queue) > 1:
            left = heapq.heappop(priority_queue)
            right = heapq.heappop(priority_queue)
            merged = Node(None, left.freq + right.freq)
            merged.left = left
            merged.right = right
            heapq.heappush(priority_queue, merged)

        return priority_queue[0]

    def build_huffman_mapping(self, node, code=''):
        if node.char is not None:
            self.codes[node.char] = code
        else:
            self.build_huffman_mapping(node.left, code + '0')
            self.build_huffman_mapping(node.right, code + '1')

    def get_encoded_text(self):
        return ''.join(self.codes[char] for char in self.text)

    def calculate_metrics(self):
        original_message=self.text
        encoded_text = self.get_encoded_text()
        initial_bits = len(self.text) * 8
        final_bits = len(encoded_text)
        compression_ratio = round(initial_bits / final_bits if final_bits else 1,3)
        probabilities = {char: round(self.text.count(char) / len(self.text),2) for char in set(self.text)}
        entropy = -sum(p * math.log2(p) for p in probabilities.values())
        avg_length = round(sum(len(self.codes[char]) * self.text.count(char) / len(self.text) for char in set(self.text)),3)
        efficiency = round(entropy / avg_length,3) if avg_length else 0

        return {
            "original message":original_message,
            "Initial Bits": initial_bits,
            "Entropy": entropy,
            "Encoded Text": encoded_text,
            "Final Bits": final_bits,
            "Probabilities": probabilities,
            "Compression Ratio": compression_ratio,
            "Average Length": avg_length,
            "Efficiency": efficiency,
            'before': initial_bits
        }

## 03 Arithmetic encoding

In [4]:
getcontext().prec = 50  # Set precision to a high value

def tag_to_bits(tag, precision):
    """
    Convert a floating-point tag to a binary bit sequence.
    :param tag: A floating-point number between 0 and 1.
    :param precision: The number of bits of precision to represent the tag.
    :return: A binary string representing the tag.
    """
    if not (0 <= tag < 1):
        raise ValueError("Tag must be a floating-point number between 0 and 1.")

    bits = []
    current_tag = tag
    lower_bound = Decimal(0.0)
    upper_bound = Decimal(1.0)

    for _ in range(precision):
        # Find the midpoint
        midpoint = (lower_bound + upper_bound) / Decimal(2)

        if current_tag < midpoint:
            # Tag is in the lower half
            bits.append("0")
            upper_bound = midpoint
        else:
            # Tag is in the upper half
            bits.append("1")
            lower_bound = midpoint

    return bits


class ArithmeticEncoder:
    def __init__(self, sequence):
        self.sequence = sequence

    def character_probability(self):
        char_count = {}
        for char in self.sequence:
            if char in char_count:
                char_count[char] += Decimal(1.0)
            else:
                char_count[char] = Decimal(1.0)

        total_characters = Decimal(len(self.sequence))

        char_probability = {}
        for char, count in char_count.items():
            probability = count / total_characters
            char_probability[char] = round(probability, 6)

        return char_probability

    def arithmetic_encoding(self):
        probabilities = self.character_probability()
        symbols = [k for k in probabilities.keys()]
        cumulative_probabilities = {}
        cumulative = Decimal(0.0)
        for symbol, prob in probabilities.items():
            cumulative_probabilities[symbol] = (cumulative, cumulative + prob)
            cumulative += prob
        lower = Decimal(0.0)
        upper = Decimal(1.0)
        for symbol in self.sequence:
            Range = upper - lower
            low, high = cumulative_probabilities[symbol]
            upper = lower + Range * high
            lower = lower + Range * low
        return lower, upper

    def calculate_metrics(self):
        encoder = self.arithmetic_encoding()
        original_message = self.sequence
        encoded_range = encoder
        encoded_tag = sum(encoder) / Decimal(2)
        initial_bits = len(self.sequence) * Decimal(8)
        final_bits = math.ceil(math.log2(1 / (encoder[1] - encoder[0]))) + 1  # after
        compression_ratio = round(initial_bits / final_bits, 3)
        probabilities = self.character_probability()
        entropy = sum((-pi * Decimal(math.log2(float(pi)))) for pi in probabilities.values())
        Average_Length = entropy + (Decimal(2) / Decimal(len(self.sequence)))
        Efficiency = (entropy / Average_Length) 

        return {
            "Original Message": original_message,
            "before Bits": initial_bits,
            "Entropy": entropy,
            "Encoded Range": encoded_range,
            "Encoded Tag": encoded_tag,
            "Tag in bits": "0." + ("".join(tag_to_bits(encoded_tag, final_bits))),
            "Final Bits": final_bits,
            "Compression Ratio": compression_ratio,
            "Average Length": Average_Length,
            "Efficiency": Efficiency
        }

## 04 GolombCode

In [5]:
class GolombCodeEncoder:
    def __init__(self, n):
        if all(c.isdigit() for c in n):
            self.isnumber = True
            self.n = int(n)
            self.M = round(math.sqrt(self.n))
        else:
            self.isnumber = False

    def golomb_code(self):
        quotient = self.n // self.M
        remainder = self.n % self.M

        unary = '1' * quotient + '0'

        binary_length = self.M.bit_length() - 1
        binary = format(remainder, f'0{binary_length}b')

        return unary + binary

    def calculate_metrics(self):
        if self.isnumber:
            encoded = self.golomb_code()

            def entropy_calc(p):
                return -p * math.log2(p)

            the_value_to_be_compressed = str(self.n)
            before_bits = len(the_value_to_be_compressed) * 8 # Assuming ASCII encoding
            after_bits = len(encoded)

            probabilities = {}
            message_length = len(the_value_to_be_compressed)

            for digit in the_value_to_be_compressed:
                prob = the_value_to_be_compressed.count(digit) / message_length
                probabilities[digit] = round(prob, 2)

            entropy = sum(entropy_calc(p) for p in probabilities.values())
            avg_length = round(len(encoded) / (encoded.count('1') + encoded.count('0')), 3)
            efficiency = round(entropy / avg_length, 3)

            return {
                "Original Message": the_value_to_be_compressed,
                "Initial Bits": before_bits,
                "Entropy": entropy,
                "Encoded Message": encoded,
                "Final Bits": after_bits,
                "Probabilities": probabilities,
                "Compression Ratio": round(before_bits / after_bits, 3),
                "Average Length": avg_length,
                "Efficiency": efficiency
            }
        else:
            return {
                "Original Message": "Not for this technique",
                "Initial Bits": "Not for this technique",
                "Entropy": "Not for this technique",
                "Encoded Message": "Not for this technique",
                "Final Bits": "Not for this technique",
                "Probabilities": "Not for this technique",
                "Compression Ratio": "Not for this technique",
                "Average Length": "Not for this technique",
                "Efficiency": "Not for this technique"
            }



## 05 lzw

In [6]:
class LZWEncoder:
    
    def __init__(self, data):
        self.data = data
        self.characters_list = list(data)
        self.char_ascii_dict = {}
        self.tag = []
        self.initialize_dictionary()
    
    def initialize_dictionary(self):
        # Iterate through ASCII values from 0 to 127
        for ascii_value in range(128):
            # Convert ASCII value to character
            character = chr(ascii_value)
            # Map character to its ASCII value in the dictionary
            self.char_ascii_dict[character] = ascii_value
    
    def encode(self):
        # Initialize the current_code to start from 128
        current_code = 128
        # Initialize the current_string with the first character from the characters_list
        current_string = self.characters_list[0]
        # Iterate through the characters_list starting from the second character
        for char in self.characters_list[1:]:
            # Append the current character to the current_string
            new_string = current_string + char
            # If the new_string is in the dictionary
            if new_string in self.char_ascii_dict:
                # Update the current_string to the new_string
                current_string = new_string
            else:
                # Append the code for the current_string to the tag list
                self.tag.append(self.char_ascii_dict[current_string])
                # Add the new_string to the dictionary with a new code
                self.char_ascii_dict[new_string] = current_code
                # Increment the current_code
                current_code += 1
                # Update the current_string to the current character
                current_string = char
        # Append the code for the last current_string to the tag list
        code = self.tag.append(self.char_ascii_dict[current_string])
        
        return code
    
    def calculate_metrics(self):
        original_message = self.data
        self.encode()
        # each char in ascii code is 8 bits
        bits_before = (len(self.data) * 8)  
        # Initialize a list to store the lengths of codewords
        codeword_lengths = []
        # Iterate through each codeword and store its length
        for codeword in self.tag:
            # Convert the codeword to binary and get its length
            binary_length = len(bin(codeword).replace("0b", ""))
            # Append the binary length to the list
            codeword_lengths.append(binary_length)
        # Calculate the maximum length among all codewords
        max_length = max(codeword_lengths)
        # calc bits after
        bits_after = len(self.tag) * max_length
        # calc compression ratio
        compression_ratio =round(bits_before / bits_after,3)
        
        # calc char prob
        character_count = {}
        for char in self.data:
            if char in character_count:
                character_count[char] += 1
            else:
                character_count[char] = 1
        
        total_characters = len(self.data)
        character_probability = {char: count / total_characters for char, count in character_count.items()}
        # calc entropy
        entropy = sum(-p * math.log2(p) for p in character_probability.values())
        
        # calc Average Length
        # Calculate the number of unique codewords
        codeword_count = len(self.tag)
        # Initialize dictionaries to store probabilities and bit lengths for each codeword
        codeword_probabilities = {}
        codeword_bit_lengths ={}
        # Iterate through each unique codeword
        for codeword in set(self.tag):
            # Calculate the probability of occurrence of the current codeword
            codeword_probabilities[codeword] = self.tag.count(codeword) / codeword_count
            # Calculate the probability of occurrence of the current codeword
            codeword_bit_lengths[codeword] = len(bin(codeword)) - 2
        # Calculate the average length (Lavg) of codewords by summing the product of each codeword's probability and its bit length
        Lavg = round(sum(probability * bit_length for codeword, probability in codeword_probabilities.items() for bit_length in [codeword_bit_lengths[codeword]]),3)
        
        # clac effeciency 
        efficiency = round(entropy/Lavg,3)
        
        return {
            "Original Message": original_message,
            "Initial Bits": bits_before,
            "Entropy": entropy,
            "Encoded message": self.tag,
            "Final Bits": bits_after,
            "Probabilities": character_probability,
            "Compression Ratio": compression_ratio,
            "Average Length": Lavg,
            "Efficiency": efficiency
        }

# MAIN

### best compression technique

In [7]:
def compare_compression_ratios(RLE_metrics, huffman_metrics, arithmetic_metrics, golomb_metrics, LZW_metrics):
    compression_ratios = {
        "RLE": RLE_metrics["Compression Ratio"],
        "Huffman": huffman_metrics["Compression Ratio"],
        "Arithmetic": arithmetic_metrics["Compression Ratio"],
        "LZW": LZW_metrics["Compression Ratio"]
    }

    if isinstance(golomb_metrics["Compression Ratio"], (int, float)):
        compression_ratios["Golomb"] = golomb_metrics["Compression Ratio"]

    best_algorithm = max(compression_ratios, key=compression_ratios.get)
    return f"The best algorithm is : {best_algorithm},with a compression ratio of {compression_ratios[best_algorithm]:.2f}"


### Run All

In [8]:
data = "awdwadawdgemonaoiwhdlnwadm[ja]"

print("------------------------RLE------------------------")
RLE_encoder = RLEEncoder(data)
RLE_metrics = RLE_encoder.calculate_metrics()
print('\n'.join([f'{key} : {value}' for key, value in list(RLE_metrics.items())[3:]]))
print("------------------------Huffman------------------------")
huffman_encoder = HuffmanCoding(data)
huffman_metrics = huffman_encoder.calculate_metrics()
print('\n'.join([f'{key} : {value}' for key, value in list(huffman_metrics.items())[3:]]))
print("------------------------Arithemtic------------------------")
arithmetic_encoder = ArithmeticEncoder(data)
arithmetic_metrics = arithmetic_encoder.calculate_metrics()
print('\n'.join([f'{key} : {value}' for key, value in list(arithmetic_metrics.items())[3:]]))
print("------------------------GLOMBCODE------------------------")
golomb_encoder = GolombCodeEncoder(data)
golomb_metrics = golomb_encoder.calculate_metrics()
print('\n'.join([f'{key} : {value}' for key, value in list(golomb_metrics.items())[3:]]))
print("------------------------LZW------------------------")
LZW_encoder = LZWEncoder(data)
LZW_metrics = LZW_encoder.calculate_metrics()
print('\n'.join([f'{key} : {value}' for key, value in list(LZW_metrics.items())[3:]]))
print("------------------------best_technique------------------------")
print(compare_compression_ratios(RLE_metrics, huffman_metrics, arithmetic_metrics, golomb_metrics, LZW_metrics))


------------------------RLE------------------------
Encoded Seq : (a, 1)(w, 1)(d, 1)(w, 1)(a, 1)(d, 1)(a, 1)(w, 1)(d, 1)(g, 1)(e, 1)(m, 1)(o, 1)(n, 1)(a, 1)(o, 1)(i, 1)(w, 1)(h, 1)(d, 1)(l, 1)(n, 1)(w, 1)(a, 1)(d, 1)(m, 1)([, 1)(j, 1)(a, 1)(], 1)
Final Bits : 270
Probabilities : {'a': 0.2, 'w': 0.17, 'd': 0.17, 'g': 0.03, 'e': 0.03, 'm': 0.07, 'o': 0.07, 'n': 0.07, 'i': 0.03, 'h': 0.03, 'l': 0.03, '[': 0.03, 'j': 0.03, ']': 0.03}
Compression Ratio : 0.889
------------------------Huffman------------------------
Encoded Text : 01111110111011100111111010101001001001100000001100010100111001011100011100011101110100100110101110110110
Final Bits : 104
Probabilities : {'w': 0.17, 'g': 0.03, 'n': 0.07, 'i': 0.03, 'e': 0.03, 'm': 0.07, 'h': 0.03, 'l': 0.03, 'j': 0.03, 'o': 0.07, 'a': 0.2, 'd': 0.17, ']': 0.03, '[': 0.03}
Compression Ratio : 2.308
Average Length : 3.467
Efficiency : 0.967
before : 240
------------------------Arithemtic------------------------
Encoded Range : (Decimal('0.053402981

## New GUI

In [9]:
window = tk.Tk()

window.geometry("1500x720")
window.configure(bg = "#FFFFFF")

message=""
def submit_handler():
    global message
    taken_message = entry_1.get()
    message=taken_message
    # ------------------------RLE------------------------
    RLE_encoder = RLEEncoder(message)
    RLE_metrics = RLE_encoder.calculate_metrics()
    # ------------------------Huffman------------------------
    huffman_encoder = HuffmanCoding(message)
    huffman_metrics = huffman_encoder.calculate_metrics()
    # ------------------------Arithemtic------------------------
    arithmetic_encoder = ArithmeticEncoder(message)
    arithmetic_metrics = arithmetic_encoder.calculate_metrics()
    # ------------------------GLOMBCODE------------------------
    golomb_encoder = GolombCodeEncoder(message)
    golomb_metrics = golomb_encoder.calculate_metrics()
    # ------------------------LZW------------------------
    LZW_encoder = LZWEncoder(message)
    LZW_metrics = LZW_encoder.calculate_metrics()
    # ------------------------------------------------
    canvas.itemconfig(tagOrId=original_message,text = "Original Message :"+message)
    canvas.itemconfig(tagOrId=initial_bits,text = "Bits before compression :"+str(RLE_metrics["Initial Bits"]))
    canvas.itemconfig(tagOrId=entropy,text = "Entropy :"+str(RLE_metrics["Entropy"])+"bits/symbol")

    RLE_data_text.delete(1.0, tk.END)
    RLE_data_text.insert(tk.INSERT, '\n'.join([f'{key} : {value}' for key, value in list(RLE_metrics.items())[3:]]))

    Huffman_data_text.delete(1.0, tk.END)
    Huffman_data_text.insert(tk.INSERT, '\n'.join([f'{key} : {value}' for key, value in list(huffman_metrics.items())[3:]]))

    Arithemtic_data_text.delete(1.0, tk.END)
    Arithemtic_data_text.insert(tk.INSERT, '\n'.join([f'{key} : {value}' for key, value in list(arithmetic_metrics.items())[3:]]))

    Golomb_data_text.delete(1.0, tk.END)
    Golomb_data_text.insert(tk.INSERT, '\n'.join([f'{key} : {value}' for key, value in list(golomb_metrics.items())[3:]]))

    LZW_data_text.delete(1.0, tk.END)
    LZW_data_text.insert(tk.INSERT, '\n'.join([f'{key} : {value}' for key, value in list(LZW_metrics.items())[3:]]))
    
    canvas.itemconfig(tagOrId=best_technique,text = compare_compression_ratios(RLE_metrics, huffman_metrics, arithmetic_metrics, golomb_metrics, LZW_metrics))


canvas = Canvas(
    window,
    bg = "#FFFFFF",
    height = 720,
    width = 1500,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge"
)

canvas.place(x = 0, y = 0)
image_image_1 = PhotoImage(file=(Path(r"assets\frame0\image_1.png")))
image_1 = canvas.create_image(
    750.0,
    55.99999999999997,
    image=image_image_1
)

canvas.create_text(
    421.0,
    115.99999999999997,
    anchor="nw",
    text="Enter The Message:",
    fill="#000000",
    font=("JosefinSansRoman Regular", 32 * -1)
)

entry_image_1 = PhotoImage(file=(Path(r"assets\frame0\entry_1.png")))
entry_bg_1 = canvas.create_image(
    850.0,
    133.49999999999997,
    image=entry_image_1
)
entry_1 = Entry(
    bd=0,
    bg="#D9D9D9",
    fg="#000716",
    highlightthickness=0
)
entry_1.place(
    x=735.0,
    y=111.99999999999997,
    width=230.0,
    height=41.0
)

canvas.create_text(
    514.0,
    112.99999999999997,
    anchor="nw",
    text=" ",
    fill="#000000",
    font=("JosefinSansRoman Regular", 32 * -1)
)

button_image_1 = PhotoImage(file=(Path(r"assets\frame0\button_1.png")))
button_1 = Button(
    image=button_image_1,
    borderwidth=0,
    highlightthickness=0,
    command=submit_handler,
    relief="flat"
)
button_1.place(
    x=981.0,
    y=109.99999999999997,
    width=143.0,
    height=48.0
)

canvas.create_text(
    640.0,
    189.99999999999997,
    anchor="nw",
    text="General Informations",
    fill="#000000",
    font=("JosefinSansRoman Bold", 21 * -1)
)

canvas.create_rectangle(
    0,
    196.99999999999983,
    630.0,
    201.99999999999997,
    fill="#000000",
    outline="")

canvas.create_rectangle(
    845.0,
    196.9999536659246,
    1500.0,
    201.99999999999997,
    fill="#000000",
    outline="")

original_message = canvas.create_text(
    600.0,
    223.99999999999997,
    anchor="nw",
    text="Original Message :\n\n",
    fill="#000000",
    font=("JosefinSansRoman Bold", 24 * -1)
)

initial_bits = canvas.create_text(
    80.0,
    271.0,
    anchor="nw",
    text="Bits before compression : ",
    fill="#000000",
    font=("JosefinSansRoman Bold", 24 * -1)
)

entropy = canvas.create_text(
    1050.0,
    271.0,
    anchor="nw",
    text="Entropy :  bits/symbol ",
    fill="#000000",
    font=("JosefinSansRoman Bold", 24 * -1)
)

canvas.create_text(
    644.0,
    360.0,
    anchor="nw",
    text="Lossless Algorithms",
    fill="#000000",
    font=("JosefinSansRoman Bold", 21 * -1)
)

canvas.create_rectangle(
    0,
    366.9999999999999,
    635.0,
    372.0,
    fill="#000000",
    outline="")

canvas.create_rectangle(
    840.0,
    366.9999532288107,
    1500.0,
    371.99999999999994,
    fill="#000000",
    outline="")

canvas.create_text(
    53.0,
    389.0,
    anchor="nw",
    text="Run Length\nAlgorithm",
    fill="#000000",
    font=("JosefinSansRoman Bold", 20 * -1)
)

canvas.create_text(
    1207.0,
    393.0,
    anchor="nw",
    text="LZW Algorithm",
    fill="#000000",
    font=("JosefinSansRoman Bold", 20 * -1)
)

canvas.create_text(
    316.0,
    393.0,
    anchor="nw",
    text="Huffman Algorithm",
    fill="#000000",
    font=("JosefinSansRoman Bold", 20 * -1)
)

canvas.create_text(
    580.0,
    393.0,
    anchor="nw",
    text="Arithmetic Algorithm",
    fill="#000000",
    font=("JosefinSansRoman Bold", 20 * -1)
)

canvas.create_text(
    944.0,
    389.0,
    anchor="nw",
    text="Golomb Algorithm",
    fill="#000000",
    font=("JosefinSansRoman Bold", 20 * -1)
)

canvas.create_rectangle(
    258.0,
    450.0,
    260.0,
    587.0,
    fill="#000000",
    outline="")

canvas.create_rectangle(
    518.0,
    450.0,
    520.0,
    587.0,
    fill="#000000",
    outline="")

canvas.create_rectangle(
    880.0,
    450.0,
    882.0,
    587.0,
    fill="#000000",
    outline="")

canvas.create_rectangle(
    1143.0,
    450.0,
    1145.0,
    587.0,
    fill="#000000",
    outline="")

RLE_data_frame = tk.Frame(window, bg="#FFFFFF")
RLE_data_frame.place(x=11.0, y=459.0, width=240, height=150)
RLE_data_text = tk.Text(RLE_data_frame, width=30, height=7, font=("JosefinSansRoman Bold", 16 * -1), wrap=tk.NONE)
RLE_data_text.grid(row=0, column=0, sticky="nsew")
RLE_data_scrollbar = tk.Scrollbar(RLE_data_frame, orient=tk.HORIZONTAL, bg="#FFFFFF", width=0)
RLE_data_scrollbar.grid(row=1, column=0, sticky="ew")
RLE_data_text.config(xscrollcommand=RLE_data_scrollbar.set)
RLE_data_scrollbar.config(command=RLE_data_text.xview)
RLE_data_frame.rowconfigure(0, weight=1)
RLE_data_frame.columnconfigure(0, weight=1)

Huffman_data_frame = tk.Frame(window, bg="#FFFFFF")
Huffman_data_frame.place(x=273.0, y=457.0, width=240, height=150)
Huffman_data_text = tk.Text(Huffman_data_frame, width=30, height=7, font=("JosefinSansRoman Bold", 16 * -1), wrap=tk.NONE)
Huffman_data_text.grid(row=0, column=0, sticky="nsew")
Huffman_data_scrollbar = tk.Scrollbar(Huffman_data_frame, orient=tk.HORIZONTAL, bg="#FFFFFF", width=0)
Huffman_data_scrollbar.grid(row=1, column=0, sticky="ew")
Huffman_data_text.config(xscrollcommand=Huffman_data_scrollbar.set)
Huffman_data_scrollbar.config(command=Huffman_data_text.xview)
Huffman_data_frame.rowconfigure(0, weight=1)
Huffman_data_frame.columnconfigure(0, weight=1)

Arithemtic_data_frame = tk.Frame(window, bg="#FFFFFF")
Arithemtic_data_frame.place(x=534.0, y=457.0, width=240, height=150)
Arithemtic_data_text = tk.Text(Arithemtic_data_frame, width=30, height=7, font=("JosefinSansRoman Bold", 16 * -1), wrap=tk.NONE)
Arithemtic_data_text.grid(row=0, column=0, sticky="nsew")
Arithemtic_data_scrollbar = tk.Scrollbar(Arithemtic_data_frame, orient=tk.HORIZONTAL, bg="#FFFFFF", width=0)
Arithemtic_data_scrollbar.grid(row=1, column=0, sticky="ew")
Arithemtic_data_text.config(xscrollcommand=Arithemtic_data_scrollbar.set)
Arithemtic_data_scrollbar.config(command=Arithemtic_data_text.xview)
Arithemtic_data_frame.rowconfigure(0, weight=1)
Arithemtic_data_frame.columnconfigure(0, weight=1)

Golomb_data_frame = tk.Frame(window, bg="#FFFFFF")
Golomb_data_frame.place(x=895.0, y=452.0, width=240, height=150)
Golomb_data_text = tk.Text(Golomb_data_frame, width=30, height=7, font=("JosefinSansRoman Bold", 16 * -1), wrap=tk.NONE)
Golomb_data_text.grid(row=0, column=0, sticky="nsew")
Golomb_data_scrollbar = tk.Scrollbar(Golomb_data_frame, orient=tk.HORIZONTAL, bg="#FFFFFF", width=0)
Golomb_data_scrollbar.grid(row=1, column=0, sticky="ew")
Golomb_data_text.config(xscrollcommand=Golomb_data_scrollbar.set)
Golomb_data_scrollbar.config(command=Golomb_data_text.xview)
Golomb_data_frame.rowconfigure(0, weight=1)
Golomb_data_frame.columnconfigure(0, weight=1)

LZW_data_frame = tk.Frame(window, bg="#FFFFFF")
LZW_data_frame.place(x=1157.0, y=452.0, width=240, height=150)
LZW_data_text = tk.Text(LZW_data_frame, width=30, height=7, font=("JosefinSansRoman Bold", 16 * -1), wrap=tk.NONE)
LZW_data_text.grid(row=0, column=0, sticky="nsew")
LZW_data_scrollbar = tk.Scrollbar(LZW_data_frame, orient=tk.HORIZONTAL, bg="#FFFFFF", width=0)
LZW_data_scrollbar.grid(row=1, column=0, sticky="ew")
LZW_data_text.config(xscrollcommand=LZW_data_scrollbar.set)
LZW_data_scrollbar.config(command=LZW_data_text.xview)
LZW_data_frame.rowconfigure(0, weight=1)
LZW_data_frame.columnconfigure(0, weight=1)

canvas.create_text(
    665.0,
    609.0,
    anchor="nw",
    text="Best technique",
    fill="#000000",
    font=("JosefinSansRoman Bold", 21 * -1)
)

canvas.create_rectangle(
    0,
    615.9999999999999,
    650.0,
    621.0,
    fill="#000000",
    outline="")

canvas.create_rectangle(
    826.0,
    615.9999519174689,
    1500,
    621.0,
    fill="#000000",
    outline="")

best_technique = canvas.create_text(
    350.0,
    652.0,
    anchor="nw",
    text="The best compression techniques is : ,having compression Ratio : ",
    fill="#000000",
    font=("JosefinSansRoman Bold", 24 * -1)
)
window.resizable(False, False)
window.mainloop()